In [340]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
import functools

In [26]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [386]:
possible_plans = [
        ['orange','black','green'],
        ['blue','black','yellow'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [660]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
    
    cubes:
    -1 - none
    
"""
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
        
    def __str__(self):
        return str(self.state)
    
    def gen_reversed_path(self):
        sn = self
        while sn != None:
            yield sn
            sn = sn.upper
    
    def get_path(self):
        return reversed(list(self.gen_reversed_path()))

class State():
    X = [0,1,2]
    profile_reduce = {
        0:0,
        1:1,
        2:0,
        3:0,
        4:4,
        -1:0
    }
    def __init__(self, lines, reversed_plan = False, profile = [0,0,0], heights = [0,0,0], x = 0, y = 0, time = 0):
        self.lines = lines
        self.heights = heights
        self.profile = profile
        self.x = x
        self.y = y
        self.time = time
        self.n_picks = 0
        self.picked_sequence = False
        self.reversed_plan = reversed_plan
    
    def get_next_allowed_cube_by_profile(self, man):
        if self.profile[man] in [-1,0]:
            return 1
        elif self.profile[man] == 3:
            return 0
        elif self.profile[man] == 4:
            return 4
        else:
            return self.profile[man] + 1
    
    def can_pick_magic(self, man, i, num):
        c = self.profile[man]
        if man == i:
            # here we should collect proper colors
            use_of_magic_cube    = c in [1,4] and num in [0,1,2] and np.any(self.lines==1)
            next_in_sequence     = c in [1,2] and c+1 == num 
            not_in_sequence      = c == 0 and num in [0,1]
            finished_sequence    = self.picked_sequence == True
            if use_of_magic_cube or next_in_sequence or not_in_sequence or finished_sequence:
                return True
        else:
            # for others:
            unused_cube          = num == 0
            used_magic_cube      = c in [1,4] and num in [0,1] and np.any(self.lines==1)
            finished_sequence    = self.picked_sequence == True
            if unused_cube or used_magic_cube or finished_sequence:
                return True
        return False
    
    def pick_cube(self, man, i, x, y):
        self.heights[i] += 1 # increase height of i-th man
        self.profile[i] = self.lines[x][i+y]
        self.lines[x][i+y] = -1 # remove cube from heap
        if self.profile[man] == 3:
            self.picked_sequence = True
        
        
    def get_next_allowed_cube(self):
        a = 4
        for l in self.lines:
            for c in l:
                if c > 0:
                    a = min(c, a)
        # return 4 if whole plan picked
        return a
    
    def get_next_line(self):
        for x, l in zip(self.X, self.lines):
            any_cube = False
            for c in l:
                if c != -1:
                    any_cube = True
            if any_cube:
                return x,l
            
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.lines:
            s += str(l) + '\n'
        s += "x=%d, y=%d, time=%f, %d picks total\n"%(self.x, self.y, self.time, self.n_picks)
        s += "height = " + str(self.heights) + '\n'
        s += "profile = " + str(self.profile) + '\n'
        s += '-----------'
        return s
    
    def get_key(self):
        return str(self.heights + [self.profile_reduce[p] for p in self.profile])
    
    def is_final(self):
        # if all is -1
        return np.all(self.lines < 0)
    
    def is_subfinal(self):
        return self.picked_sequence
    
class SuctionRobotStrategyOptimizer():
    defaults = {
        'plan': [0,1,2],
        'allowed_sides': [
            [False, True, True, True],
            [True, False, True, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, True, False]
        ],
        'heaps': [0,1,2],
        'pick_time': 2,
        'move_time': 1,
        'heap': {'orange':np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ]),
                 'green':np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ])
            }
    }
    def __init__(self, **kvargs):
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
        self.manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
        
    def rotate_heap(self, heap, side):
        return np.rot90(heap, side)
    
    def get_heap(self, number = 0, side = 0):
        heap = None
        if number in [0,1,2]:
            heap = self.params['heap']['orange']
        if number in [3,4,5]:
            heap = self.params['heap']['green']
        return self.rotate_heap(heap,side)
    
    def set_plan(self, plan):
        if isinstance(plan[0], str):
            self.params['plan'] = [colormap[c] for c in plan]
        else:
            self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num in [0,1,2]:
            return self.params['plan'][num]
        return self.params['plan']
    
    def reverse_plan(self):
        self.params['plan'] = self.params['plan'][::-1]
        
    def get_move_time(self, dx, dy):
        return max(abs(dx),abs(dy))*self.params['move_time']
    
    def get_lines_by_plan_and_heap(self, heap):
        lines = copy.copy(heap)
        for i, j in iterer(3,3):
            if heap[i][j] in range(5):
                if heap[i][j] in self.get_plan():
                    lines[i][j] = self.get_plan().index(heap[i][j]) + 1
                else:
                    lines[i][j] = 0
        return lines
    
    def try_to_pick(self, state, y, l, c, x, man, mans):
        picked = [-1,-1,-1]
        for i in range(3): # run over mans
            if mans[i] == 1: # if need to pick by i-th man
                if i + y in [0,1,2] and l[i+y] >= 0: # check if we pick existing cube
                    # print(i,y)
                    if (i == man and (c == l[i+y] or (c in [1,4] and l[i+y]==0))) or (i != man and l[i+y] == 0) : # if we can pick this cube according to plan 
                        picked[i] = l[i+y] # add to picked (why?)
                        state.heights[i] += 1 # increase height of i-th man
                        state.lines[x][i+y] = -1 # remove cube from heap
                        # print(i, y, i+y)
                    else:
                        return None
                else:
                    return None
        # if still not return, add time and return new state
        state.time += self.params['pick_time']
        state.time += self.get_move_time(x-state.x, y-state.y)
        state.n_picks += 1
        state.x = x
        state.y = y
        return state
    
    def try_to_pick_magic(self, state, y, l, x, man, mans):
        for i in range(3): # run over mans
            if mans[i] == 1: # if need to pick by i-th man
                if i + y in [0,1,2] and l[i+y] >= 0 and state.can_pick_magic(man, i, l[i+y]): # check if we pick existing cube
                    state.pick_cube(man, i, x ,y)
                else:
                    return None
        # if still not return, add time and return new state
        state.time += self.params['pick_time']
        state.time += self.get_move_time(x-state.x, y-state.y)
        state.n_picks += 1
        state.x = x
        state.y = y
        return state
    
    def get_new_states(self, state, man):
        if not state.is_final():
            c = state.get_next_allowed_cube()
            x, l = state.get_next_line()
            for mans, y in iterer(self.manipulator_masks, range(-2,3)):
                new_state = self.try_to_pick(copy.deepcopy(state), y, l, c, x, man, mans)
                if new_state != None:
                    # print(y, mans)
                    yield new_state
                    
    def get_new_states_magic(self, state, man):
        if not state.is_final():
            x, l = state.get_next_line()
            for mans, y in iterer(self.manipulator_masks, range(-2,3)):
                new_state = self.try_to_pick_magic(copy.deepcopy(state), y, l, x, man, mans)
                if new_state != None:
                    # print(y, mans)
                    yield new_state
                    
    def reduce_final_states(self, final_states):
        # we can choose only same heights with best times
        heights = {}
        for sn in final_states:
            s = sn.state
            key = str(s.heights)
            if key in heights:
                if heights[key].state.time > s.time:
                    heights[key] = sn
            else:
                heights[key] = sn
        return list(heights.values())
    
    def reduce_final_states_magic(self, final_states):
        heights_and_profiles = {}
        for sn in final_states:
            s = sn.state
            key = s.get_key()
            if key in heights_and_profiles:
                if heights_and_profiles[key].state.time > s.time:
                    heights_and_profiles[key] = sn
            else:
                heights_and_profiles[key] = sn
        return list(heights_and_profiles.values())
    
    def get_all_profiles(self, final_states):
        profiles = {}
        for sn in final_states:
            s = sn.state
            key = str(s.profile)
            if key in profiles:
                if profiles[key].state.time > s.time:
                    profiles[key] = sn
            else:
                profiles[key] = sn
        return list(profiles.keys())
    
    def build_solution_tree(self, heap_num, side, man):
        h = self.get_heap(heap_num, side)
        initial_state = State(opt.get_lines_by_plan_and_heap(h))
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        N = 0
        final_states = []
        while len(this_wave) > 0:
            for node in this_wave:
                if node.state.is_final():
                    final_states.append(node)
                node.nexts = [STNode(x,node) for x in self.get_new_states(node.state, man)]
                next_wave.extend(node.nexts)
            if len(next_wave) > 0:
                N = len(next_wave)
            this_wave = next_wave
            next_wave = []
        return root, self.reduce_final_states(final_states)
    
    def build_solution_tree_magic(self, heap_num, side, man, profiles=[0,0,0], reversed_plan=False):
        h = self.get_heap(heap_num, side)
        heights = [1 if p > 0 else 0 for p in profiles]
        initial_state = State(opt.get_lines_by_plan_and_heap(h),reversed_plan, profiles, heights)
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        N = 0
        final_states = []
        while len(this_wave) > 0:
            for node in this_wave:
                if node.state.is_subfinal():
                    final_states.append(node)
                node.nexts = [STNode(x,node) for x in self.get_new_states_magic(node.state, man)]
                next_wave.extend(node.nexts)
            if len(next_wave) > 0:
                N = len(next_wave)
            this_wave = next_wave
            next_wave = []
        return root, self.reduce_final_states_magic(final_states)
    
    def count_all_variants(self, **kvargs):
        team_colors, sides, mans = 2, 4, 3
        if 'heap_num' in kvargs:
            team_colors = [kvargs['heap_num'] // 3]
        if 'use_sides_constraint' in kvargs and kvargs['use_sides_constraint'] and 'heap_num' in kvargs:
            sides = [side for allowed, side in zip(self.params['allowed_sides'][kvargs['heap_num']],list(range(4))) if allowed ]

        variants = []
        progress = 0
        for team_color, side, man in iterer(team_colors, sides, mans):
            _, final_states  = self.build_solution_tree(team_color*3, side, man)
            self.reverse_plan()
            _, final_states2 = self.build_solution_tree(team_color*3, side, man)
            final_states.extend(final_states2)
            for fs in self.reduce_final_states(final_states):
                variants.append([team_color, side, man, fs])
            progress += 1
            if 'progress_bar' in kvargs and kvargs['progress_bar']:
                print("%d%s" % (progress / (team_colors*sides*mans)*100, '%')) 
        return variants
    
    def convert_profile(self, profile, team_color,side, color_to_order=True):
        if color_to_order:
            order_profile = []
            for c in profile:
                if c == -1:
                    pass
                
    def count_all_variants_magic(self, colored_profiles=[[0,0,0]], **kvargs):
        team_colors, sides, mans = 2, 4, 3
        if 'heap_num' in kvargs:
            team_colors = [kvargs['heap_num'] // 3]
        if 'use_sides_constraint' in kvargs and kvargs['use_sides_constraint'] and 'heap_num' in kvargs:
            sides = [side for allowed, side in zip(self.params['allowed_sides'][kvargs['heap_num']],list(range(4))) if allowed ]

        variants = []
        progress = 0
        for team_color, side, man, color_profile in iterer(team_colors, sides, mans, colored_profiles):
            
            _, final_states  = self.build_solution_tree_magic(team_color*3, side, man, profile)
            self.reverse_plan()
            _, final_states2 = self.build_solution_tree(team_color*3, side, man, profile, True)
            final_states.extend(final_states2)
            for fs in self.reduce_final_states(final_states):
                variants.append([team_color, side, man, fs])
            progress += 1
            if 'progress_bar' in kvargs and kvargs['progress_bar']:
                print("%d%s" % (progress / (team_colors*sides*mans)*100, '%')) 
        return variants
    
    def count_all_picking_plans(self,heap_nums = [0,1,2], **kvargs):
        variants = [self.count_all_variants(heap_num=hn, use_sides_constraint=True) for hn in heap_nums]
        
        heights = {}
        for v in variants[-1]:
            key = str(v[-1].state.heights)
            if key in heights:
                heights[key].append(v)
            else:
                heights[key]= [v]
                    
        path_variants = []
        i = 0
        total = len(variants[0])*len(variants[1])
        for v1,v2 in iterer(variants[0],variants[1]):
            height_left = [5-h1-h2 for h1,h2 in zip(v1[-1].state.heights, v2[-1].state.heights)]
            key = str(height_left)
            if key in heights:
                for v3 in heights[key]:
                    path_variants.append([v1,v2,v3])
            i+=1
            if 'progress_bar' in kvargs and kvargs['progress_bar']:
                print("%d%s" % (i / total *100, '%')) 
        return path_variants
    
    def count_best_picking_plan(self, heap_nums = [0,1,2]):
        path_variants = self.count_all_picking_plans(heap_nums)
        min_v = path_variants[0]
        min_t = sum([v[-1].state.time for v in min_v])
        for pv in path_variants:
            pv_t = sum([v[-1].state.time for v in pv])
            if pv_t < min_t:
                min_v = pv
                min_t = pv_t
        return min_v
    
    def get_heap_path(self, variant):
        root, final_states = self.build_solution_tree(v[0]*3,v[1],v[2])
        route = []
        for fs in final_states:
            if fs.state.heights == v[-1].state.heights:
                sn = fs
                while sn != None:
                    route.append(sn)
                    sn = sn.upper
                break
        return reversed(route)
            
        

In [661]:
opt = SuctionRobotStrategyOptimizer()

In [689]:
opt.set_plan(possible_plans[2])
opt.get_plan()

[2, 1, 3]

In [696]:
testing_triple = [0,2,0]
print(opt.get_lines_by_plan_and_heap(opt.get_heap(*testing_triple[:2])))

[[-1  1 -1]
 [ 3  0  2]
 [-1  0 -1]]


In [697]:
_, final_states = opt.build_solution_tree(*testing_triple)

In [698]:
len(final_states)

6

In [699]:
for sn in final_states[2].get_path():
    print(sn.state)

---STATE---
[-1  1 -1]
[3 0 2]
[-1  0 -1]
x=0, y=0, time=0.000000, 0 picks total
height = [0, 0, 0]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[3 0 2]
[-1  0 -1]
x=0, y=1, time=3.000000, 1 picks total
height = [1, 0, 0]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[ 3  0 -1]
[-1  0 -1]
x=1, y=2, time=6.000000, 2 picks total
height = [2, 0, 0]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  0 -1]
x=1, y=0, time=10.000000, 3 picks total
height = [3, 1, 0]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=13.000000, 4 picks total
height = [4, 1, 0]
profile = [0, 0, 0]
-----------


In [ ]:
### _, final_states2 = opt.build_solution_tree(*testing_triple)
len(final_states2)

In [627]:
for fs in final_states:
    print(fs.state)

---STATE---
[-1 -1 -1]
[-1  0 -1]
[-1  1 -1]
x=1, y=1, time=10.000000, 3 picks total
height = [0, 3, 1]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
x=1, y=1, time=10.000000, 3 picks total
height = [1, 3, 1]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
x=1, y=1, time=10.000000, 3 picks total
height = [0, 3, 2]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1  0 -1]
[-1  1 -1]
x=1, y=1, time=11.000000, 3 picks total
height = [1, 3, 0]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
x=1, y=1, time=11.000000, 3 picks total
height = [2, 3, 0]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
x=1, y=0, time=13.000000, 4 picks total
height = [0, 4, 1]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=14.000000, 4 picks total
height = [1, 3, 2]
profile = [0, 3, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=

In [628]:
for fs in final_states2:
    print(fs.state)

In [511]:
opt.get_all_profiles(final_states)

['[3, 0, 0]', '[0, 0, 0]']

In [629]:
for s in final_states[-1].get_path():
    print(s.state)

---STATE---
[-1  0 -1]
[2 0 3]
[-1  1 -1]
x=0, y=0, time=0.000000, 0 picks total
height = [0, 1, 0]
profile = [0, 4, 0]
-----------
---STATE---
[-1 -1 -1]
[2 0 3]
[-1  1 -1]
x=0, y=1, time=3.000000, 1 picks total
height = [1, 1, 0]
profile = [0, 4, 0]
-----------
---STATE---
[-1 -1 -1]
[-1  0  3]
[-1  1 -1]
x=1, y=-1, time=7.000000, 2 picks total
height = [1, 2, 0]
profile = [0, 2, 0]
-----------
---STATE---
[-1 -1 -1]
[-1  0 -1]
[-1  1 -1]
x=1, y=1, time=11.000000, 3 picks total
height = [1, 3, 0]
profile = [0, 3, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
x=1, y=0, time=14.000000, 4 picks total
height = [1, 4, 0]
profile = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=17.000000, 5 picks total
height = [2, 4, 0]
profile = [1, 0, 0]
-----------


In [476]:
h = opt.get_heap(0, 0)
initial_state = State(opt.get_lines_by_plan_and_heap(h),False, [0,0,0])
for state in opt.get_new_states_magic(initial_state, 0):
    print(state)

---STATE---
[-1 -1 -1]
[0 3 0]
[-1  1 -1]
x=0, y=1, time=3.000000, 1 picks total
height = [1, 0, 0]
-----------


In [408]:
for p in possible_plans:
    opt.set_plan(p)
    times, picks, plans = [100,0], [1,1], [1,1]
    for heap_plan in itertools.permutations([0,1,2]):
        best_pv = opt.count_best_picking_plan(heap_plan)
        #for v in best_pv:
        #    print(v[:3],v[-1])
        total_time = sum([v[-1].state.time for v in best_pv])
        total_picks = sum([v[-1].state.n_picks for v in best_pv])
        if total_time < times[0]:
            times[0] = total_time
            picks[0] = total_picks
            plans[0] = heap_plan
        if total_time > times[1]:
            times[1] = total_time
            picks[1] = total_picks
            plans[1] = heap_plan
    print("plan %d %d %d | best %f %d heaps %d %d %d | worst %f %d heaps %d %d %d\n"%(p[0],p[1],p[2], times[0], picks[0], plans[0][0], plans[0][1], plans[0][1],times[1], picks[1], *plans[1]))

TypeError: %d format: a number is required, not str

In [406]:
for p in possible_plans:
    opt.set_plan(p)
    best_pv = opt.count_best_picking_plan([2,1,0])
    total_time = sum([v[-1].state.time for v in best_pv])
    total_picks = sum([v[-1].state.n_picks for v in best_pv])
    print(total_time, total_picks)

32 10
46 15
38 12
34 12
34 12
34 12
32 10
46 15
38 12
34 12


In [412]:
p = possible_plans[1]
opt.set_plan(p)
best_pv = opt.count_best_picking_plan()
for v in best_pv:
    path = opt.get_heap_path(v)
    print(v[:3])
    for s in path:
        print(s.state)
    print('----------------------------')
total_time = sum([v[-1].state.time for v in best_pv])
print(total_time)

[0, 1, 1]
---STATE---
[-1  0 -1]
[2 1 3]
[-1  0 -1]
x=0, y=0, time=0.000000, 0 picks total
height = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[2 1 3]
[-1  0 -1]
x=0, y=-1, time=3.000000, 1 picks total
height = [0, 0, 1]
-----------
---STATE---
[-1 -1 -1]
[ 2 -1  3]
[-1  0 -1]
x=1, y=0, time=6.000000, 2 picks total
height = [0, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1  3]
[-1  0 -1]
x=1, y=-1, time=9.000000, 3 picks total
height = [0, 2, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  0 -1]
x=1, y=1, time=13.000000, 4 picks total
height = [0, 3, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=16.000000, 5 picks total
height = [1, 3, 1]
-----------
----------------------------
[0, 3, 0]
---STATE---
[-1  0 -1]
[3 1 2]
[-1  0 -1]
x=0, y=0, time=0.000000, 0 picks total
height = [0, 0, 0]
-----------
---STATE---
[-1 -1 -1]
[3 1 2]
[-1  0 -1]
x=0, y=0, time=2.000000, 1 picks total
height = [0, 1, 0]
-----------
---STATE---
[-1 -1 -1]
[ 3 -1  2]
[-1  

In [326]:
for v in best_pv:
    print(v[:3],v[-1])

[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [1, 3, 1]
-----------
[0, 1, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=9.000000
height = [3, 1, 1]
-----------
[0, 3, 2] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=9.000000
height = [1, 1, 3]
-----------


In [300]:
for p in possible_plans:
    opt.set_plan(p)
    print(len(opt.count_time_matrix()))

120
132
126
132
120
120
120
96
102
120


In [295]:
for v in variants:
    print(v[:3],v[-1])

[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=9.000000
height = [3, 1, 1]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=0, time=9.000000
height = [3, 2, 0]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=11.000000
height = [3, 0, 2]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=11.000000
height = [4, 0, 1]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=0, time=11.000000
height = [4, 1, 0]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000
height = [1, 3, 1]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000
height = [2, 3, 0]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=8.000000
height = [0, 3, 2]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=10.000000
height = [0, 4, 1]
-----------
[

In [296]:
len(variants)

120

In [250]:
%%time
for str_plan in possible_plans:
    opt.set_plan(str_plan)
    root, final_states = opt.build_solution_tree(0,0,0)
    print(len(final_states))

5
6
6
5
3
6
6
5
0
6
CPU times: user 448 ms, sys: 12 ms, total: 460 ms
Wall time: 422 ms


In [241]:
opt.set_plan([colormap[c] for c in str_plan])
root, fs = opt.build_solution_tree(0,0,0)
for sn in fs:
    print(sn.state)
    
print('--------------------------------------_')
sn = fs[0]
while sn != None:
    print(sn.state)
    sn = sn.upper

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=11.000000
height = [3, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [4, 0, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [3, 2, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=9.000000
height = [4, 1, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [5, 0, 0]
-----------
--------------------------------------_
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  2 -1]
x=1, y=2, time=10.000000
height = [2, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1  1]
[-1  2 -1]
x=1, y=1, time=8.000000
height = [1, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1  3  1]
[-1  2 -1]
x=1, y=-

In [411]:
print("%d | %d"%(1,2))

1 | 2


In [413]:
list(itertools.product([0,1],repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [414]:
sn

In [417]:
str([0,1]+[1,2])

'[0, 1, 1, 2]'